In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("faresabbasai2022/skin-burn-dataset")

print("Path to dataset files:", path)

100%|██████████| 109M/109M [00:00<00:00, 148MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/faresabbasai2022/skin-burn-dataset/versions/1


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# ✅ Base path from kagglehub output
base_path = "/root/.cache/kagglehub/datasets/faresabbasai2022/skin-burn-dataset/versions/1/skin burn dataset"
train_path = os.path.join(base_path, "train")

# Image parameters
img_height, img_width = 224, 224
batch_size = 32

# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

# Train generator
train_data = datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Validation generator
val_data = datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Load base model (ResNet50)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze pretrained weights

# Build classifier on top of ResNet50
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(train_data.num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Show model summary
model.summary()

# Train the model
model.fit(train_data, epochs=10, validation_data=val_data)

# Evaluate the model
loss, acc = model.evaluate(val_data)
print(f"Validation Accuracy: {acc:.2f}")


Found 4402 images belonging to 3 classes.
Found 1098 images belonging to 3 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,113,027 (91.98 MB)

 Trainable params: 525,315 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 925s 7s/step - accuracy: 0.4205 - loss: 1.0443 - val_accuracy: 0.4344 - val_loss: 1.0173
Epoch 2/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 920s 7s/step - accuracy: 0.4390 - loss: 1.0133 - val_accuracy: 0.4189 - val_loss: 1.0019
Epoch 3/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 910s 7s/step - accuracy: 0.4320 - loss: 1.0123 - val_accuracy: 0.4481 - val_loss: 0.9810
Epoch 4/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 923s 7s/step - accuracy: 0.4655 - loss: 1.0002 - val_accuracy: 0.4490 - val_loss: 0.9710
Epoch 5/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 908s 7s/step - accuracy: 0.4565 - loss: 0.9903 - val_accuracy: 0.4481 - val_loss: 0.9598
Epoch 6/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 902s 7s/step - accuracy: 0.4475 - loss: 0.9954 - val_accuracy: 0.4636 - val_loss: 0.9522
Epoch 7/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 908s 7s/step - accuracy: 0.4822 - loss: 0.9825 - val_accuracy: 0.4727 - val_loss: 0.9481
Epoch 8/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 927s 7s/step - accuracy: 0.4688 - loss: 0.9844 - val_accu